In [14]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv('./.env')

True

## Utilizando o modelo gpt-3.5-turbo da openAPI

Esse código é só uma abstração do modelo, fazendo uma chamada para a API da openAI

In [6]:
chat = ChatOpenAI(
    model='gpt-3.5-turbo'
)

Criando uma conversa simulada com o ChatBot usando o esquema do langchain.

É necessário fazer isso porque a respota do modelo da OpenAI será baseada no histórico dessa conversa

In [7]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="Você é um assistente útil que responde perguntas."),
    HumanMessage(content="Olá Bot, como você está hoje?"),
    AIMessage(content="Estou bem, obrigado. Como posso ajudar?"),
    HumanMessage(content="Gostaria de entender o que é machine learning.")
]

Enviando a mensagem para o LLM:

In [ ]:
res = chat.invoke(messages)
res

Necessário adicionar essa mensagem no histórico de conversas:

In [ ]:
messages.append(res)

Agora tentando uma pergunta nova:

In [ ]:
prompt = HumanMessage(
    content="Qual é a diferença entre supervisionado e não supervisionado?"
)
messages.append(prompt)

Como esse tipo de modelo leva em consideração o histórico, das conversas, não precisamos mencionar a palavra "aprendizado"

In [ ]:
res = chat.invoke(messages)
print(res.content)

## Embeddings

Utilizando o modelo de embedding da OpenAI:

In [3]:
from langchain_openai import OpenAIEmbeddings
embed_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
texts = [
    'this is one chunk',
    'this is the second chunk of text'
]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

## RAG

In [ ]:
from datasets import load_dataset
dataset = load_dataset("infoslack/mistral-7b-arxiv-paper-chunked", split="train")
dataset

In [9]:
data = dataset.to_pandas()
data.head(2)

,doi,chunk-id,chunk,id,title,summary,source,authors,categories,comment,journal_ref,primary_category,published,updated,references
0,2310.06825,0,"Mistral 7B\nAlbert Q. Jiang, Alexandre Sablayr...",2310.06825,Mistral 7B,"We introduce Mistral 7B v0.1, a 7-billion-para...",http://arxiv.org/pdf/2310.06825,"[Albert Q. Jiang, Alexandre Sablayrolles, Arth...","[cs.CL, cs.AI, cs.LG]",Models and code are available at\n https://mi...,None,cs.CL,20231010,20231010,"[{'id': '1808.07036'}, {'id': '1809.02789'}, {..."
1,2310.06825,1,automated benchmarks. Our models are released ...,2310.06825,Mistral 7B,"We introduce Mistral 7B v0.1, a 7-billion-para...",http://arxiv.org/pdf/2310.06825,"[Albert Q. Jiang, Alexandre Sablayrolles, Arth...","[cs.CL, cs.AI, cs.LG]",Models and code are available at\n https://mi...,None,cs.CL,20231010,20231010,"[{'id': '1808.07036'}, {'id': '1809.02789'}, {..."


In [10]:
docs = data[['chunk', 'source']]
docs.head()

,chunk,source
0,"Mistral 7B\nAlbert Q. Jiang, Alexandre Sablayr...",http://arxiv.org/pdf/2310.06825
1,automated benchmarks. Our models are released ...,http://arxiv.org/pdf/2310.06825
2,GQA significantly accelerates the inference sp...,http://arxiv.org/pdf/2310.06825
3,Mistral 7B takes a significant step in balanci...,http://arxiv.org/pdf/2310.06825
4,parameters of the architecture are summarized ...,http://arxiv.org/pdf/2310.06825


Importando o módulo que converte o dataframe para o modelo de registro de documento do LangChain

In [11]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(docs, page_content_column="chunk")
documents = loader.load()

In [12]:
documents[0]

Document(metadata={'source': 'http://arxiv.org/pdf/2310.06825'}, page_content='Mistral 7B\nAlbert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford,\nDevendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel,\nGuillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux,\nPierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix,\nWilliam El Sayed\nAbstract\nWe introduce Mistral 7B, a 7–billion-parameter language model engineered for\nsuperior performance and efficiency. Mistral 7B outperforms the best open 13B\nmodel (Llama 2) across all evaluated benchmarks, and the best released 34B\nmodel (Llama 1) in reasoning, mathematics, and code generation. Our model\nleverages grouped-query attention (GQA) for faster inference, coupled with sliding\nwindow attention (SWA) to effectively handle sequences of arbitrary length with a\nreduced inference cost. We also provide a model fine-tuned to follow instructions,\nMistral 7B – Inst

Agora vamos utilizar o Qdrant: um mecanismo de busca por similaridade de vetores.

O qdrant configurado vai utilizar o algoritmo de embeddings da OpenAI

In [ ]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

qdrant = Qdrant.from_documents(
    documents=documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="chatbot"
)

Com o código abaixo, o qdrant irá fornecer as 3 (k=3) respostas mais similates a query passada.

Ele vai comparar a query com os chunks passados no documento anteriormente

In [ ]:
query = "O que tem de tão especial no Mistral 7B?"
qdrant.similarity_search(query, k=3)

Agora configuramos uma função do que cria um prompt customizado, incluindo os 3 trechos mais similares no documents para fornecer contexto

In [ ]:
def custom_prompt(query: str):
    results = qdrant.similarity_search(query, k=3)
    source_knowledge = "\n".join([x.page_content for x in results])
    augment_prompt = f"""Use o contexto abaixo para responder à pergunta.

    Contexto:
    {source_knowledge}

    Pergunta: {query}"""
    return augment_prompt

Avaliando o conteúdo do prompt:

In [ ]:
print(custom_prompt(query))

In [ ]:
prompt = HumanMessage(
    content=custom_prompt(query)
)

messages.append(prompt)

res = chat.invoke(messages)

print(res.content)

## Utilizando um modelo mais simples

In [ ]:
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0, model_name="llama-3.1-70b-versatile")

Configurando uma instância do modelo

In [15]:
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0, model_name="llama-3.1-70b-versatile")

In [ ]:
prompt = HumanMessage(
    content=custom_prompt(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)